In [10]:
# Dependencies
import requests
import json
import pandas as pd
import time

# Import the API key
from config import eia_key

In [11]:
#Create a set of lists to store the state, year, energy source and capacity per fuel type (in MWh)
CO2_emissions = []
years = []
sectornames = []
states_json = []

In [12]:
#Create a list of states to be used to run through in the for loop
states = ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DC", "DE", "FL",
         "GA", "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA",
         "MD", "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE",
         "NH", "NJ", "NM", "NV", "NY", "OH", "OK", "OR", "PA", "RI",
         "SC", "SD", "TN", "TX","UT", "VA", "VT", "WA", "WI", "WV", "WY"]

In [13]:
#The EIA only returns a maximum of 5000 rows per request, so this for loop makes one request per state

# Print a message to let me know the search is beginning
print("Starting State Search")

# Iterate through the list of states
for state in states:
    #For each state, let the user know when it starts searching data for that state
    print(f"Requesting data for {state}")
    
    # Set URL (taken from EIA Documentation, only changing "state" each time)
    complete_url = f"https://api.eia.gov/v2/co2-emissions/co2-emissions-aggregates/data/?frequency=annual&data[0]=value&facets[stateId][]={state}&facets[fuelId][]=TO&start=1990&end=2020&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={eia_key}"
    # Make an API request using that URL
    stateresponse = requests.get(complete_url)
    
    # Convert the API response to JSON format
    stateresponse = stateresponse.json()
    
    #The data inside the json is a dictionary, and the data about each powerplant can be found in "Response" > "Data" as a list
    #So this for loop runs through each item in the "Data" list and pulls out the necessary information
    #And appends it to the appropriate list
    for i in range(len(stateresponse["response"]["data"])):
        # Grab each state's data from the response (indexed in data) and place information in appropriate lists
        try:
            CO2_emissions.append(stateresponse["response"]["data"][i]["value"])
            sectornames.append(stateresponse["response"]["data"][i]["sector-name"])
            years.append(stateresponse["response"]["data"][i]["period"])
            states_json.append(stateresponse["response"]["data"][i]["stateId"])
        except (KeyError, IndexError):
            # If any one of these doesn't work, the row will return "No Data found"
            CO2_emissions.append("No Data found")
            sectornames.append("No Data found")
            years.append("No Data found")
            states_json.append("No Data found")
        
    # Let the user know the search for each state has been finished
    print(f"Data for {state} complete")
    
    #wait until next request to be a good person
    time.sleep(1)
    
#Finally, let the user know that the complete search has been finished
print(f"State CO2 Emissions search is complete")


Starting State Search
Requesting data for AK
Data for AK complete
Requesting data for AL
Data for AL complete
Requesting data for AR
Data for AR complete
Requesting data for AZ
Data for AZ complete
Requesting data for CA
Data for CA complete
Requesting data for CO
Data for CO complete
Requesting data for CT
Data for CT complete
Requesting data for DC
Data for DC complete
Requesting data for DE
Data for DE complete
Requesting data for FL
Data for FL complete
Requesting data for GA
Data for GA complete
Requesting data for HI
Data for HI complete
Requesting data for IA
Data for IA complete
Requesting data for ID
Data for ID complete
Requesting data for IL
Data for IL complete
Requesting data for IN
Data for IN complete
Requesting data for KS
Data for KS complete
Requesting data for KY
Data for KY complete
Requesting data for LA
Data for LA complete
Requesting data for MA
Data for MA complete
Requesting data for MD
Data for MD complete
Requesting data for ME
Data for ME complete
Requesting

In [14]:
#Combine all lists into one tidy little dataframe
CO2emissions_by_state_df = pd.DataFrame({"State" :states_json,
                           "Year":years,
                          "CO2 Emissions (Million Metric Tons)": CO2_emissions,
                          "Sector Name": sectornames})
CO2emissions_by_state_df


#Look upon your mighty dataframe and despair
CO2emissions_by_state_df

,State,Year,CO2 Emissions (Million Metric Tons),Sector Name
0,AK,2020,2.117089,Commercial carbon dioxide emissions
1,AK,2020,35.967527,Total carbon dioxide emissions from all sectors
2,AK,2020,2.735818,Electric Power carbon dioxide emissions
3,AK,2020,1.651749,Residential carbon dioxide emissions
4,AK,2020,17.589291,Industrial carbon dioxide emissions
...,...,...,...,...
9481,WY,1990,39.612559,Electric Power carbon dioxide emissions
9482,WY,1990,5.747224,Transportation carbon dioxide emissions
9483,WY,1990,0.826661,Residential carbon dioxide emissions
9484,WY,1990,10.176367,Industrial carbon dioxide emissions


In [15]:
#Then publish the mighty dataframe to a csv file to be analyzed later
CO2emissions_by_state_df.to_csv("../Output/CO2emissions_by_state.csv")